In [4]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import chess
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

NUM_SQ = 64
NUM_PT = 10
NUM_PLANES = (NUM_SQ * NUM_PT + 1)

PyTorch Version:  1.13.1+cpu
Torchvision Version:  0.14.1+cpu


In [5]:
def orient(is_white_pov: bool, sq: int):
    return (63 * (not is_white_pov)) ^ sq

def halfkp_idx(is_white_pov: bool, king_sq: int, sq: int, p: chess.Piece):
    p_idx = (p.piece_type - 1) * 2 + (p.color != is_white_pov)
    return 1 + orient(is_white_pov, sq) + p_idx * NUM_SQ + king_sq * NUM_PLANES


def get_active_features(board: chess.Board):
    def piece_features(turn):
        indices = torch.zeros(NUM_PLANES * NUM_SQ)
        for sq, p in board.piece_map().items():
            if p.piece_type == chess.KING:
                continue
            indices[halfkp_idx(turn, orient(turn, board.king(turn)), sq, p)] = 1.0
        return indices
    return (piece_features(chess.WHITE), piece_features(chess.BLACK))